You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [4]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [19]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "You are good at Korean language.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [20]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You are good at Korean language.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [21]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible."
              "You are good at Korean language.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [22]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
        "in Korean"
    ,
    agent=planner,
)

### Task: Write

In [23]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
        "in Korean",
    
    ,
    agent=writer,
)

SyntaxError: invalid syntax (1285325592.py, line 19)

### Task: Edit

In [24]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs."
                    "in Korean",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [25]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-05-20 22:48:54,290 - 22824 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [27]:
result = crew.kickoff(inputs={"topic": "맥북을 사야하는 이유"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on 맥북을 사야하는 이유.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

컨텐츠 계획서:

목차:

I. 소개
- 맥북을 사야하는 이유 소개
- 최신 동향 소개

II. 주요 내용
- 맥북의 성능 및 디자인
- 생산성 향상을 위한 맥북의 장점
- 맥북을 선택해야 하는 이유

III. 타겟 오디언스 분석
- 디자이너, 프로그래머 및 비즈니스 전문가
- 높은 생산성을 요구하는 사용자
- 기존 애플 제품을 좋아하는 소비자

IV. SEO 키워드
- 맥북, 맥북 구매, 맥북 장점, 맥북 생산성, 맥북 디자인

V. 참고 자료
1. 애플 공식 웹사이트
2. 전문 리뷰어의 맥북 리뷰
3. 유명 블로그 및 포럼 게시물

콜 투 액션:
- 맥북을 구매하고 생산성을 향상시켜보세요!

> Finished chain.
 [DEBUG]: == [Content Planner] Task output: 컨텐츠 계획서:

목차:

I. 소개
- 맥북을 사야하는 이유 소개
- 최신 동향 소개

II. 주요 내용
- 맥북의 성능 및 디자인
- 생산성 향상을 위한 맥북의 장점
- 맥북을 선택해야 하는 이유

III. 타겟 오디언스 

- Display the results of your execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction
Artificial Intelligence (AI) is a rapidly evolving field that has the potential to revolutionize various industries. Defined as the simulation of human intelligence processes by machines, AI is becoming increasingly important in sectors such as healthcare, finance, and transportation. In this blog post, we will explore the latest trends and key players in the AI industry, shedding light on the advancements that are shaping the future of technology.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the advancements in machine learning, which enable computers to learn from data and improve their performance over time. Natural language processing is also a key area of development, allowing machines to understand and generate human language. Furthermore, the integration of robotics and automation in AI systems is paving the way for more efficient and productive processes. However, as AI technologies continue to advance, ethical considerations surrounding their use become increasingly important.

## Key Players in the AI Industry
Some of the leading players in the AI industry include Google DeepMind, known for its groundbreaking research in AI and machine learning. IBM Watson is another key player, offering AI-powered solutions for businesses across various sectors. Microsoft Azure AI and Amazon Web Services AI are also prominent names in the industry, providing cloud-based AI services to organizations worldwide. These key players are driving innovation and shaping the future of AI technology.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI research include the development of advanced neural networks and deep learning algorithms. However, the increasing integration of AI in the job market raises concerns about the impact on employment opportunities. Regulatory updates in AI technology aim to address these concerns and ensure the responsible use of AI in society. Staying informed about these developments is crucial for professionals, business leaders, and researchers interested in the future of AI.

## Conclusion
In conclusion, Artificial Intelligence is a rapidly evolving field with the potential to transform industries and improve efficiency. By staying updated on the latest trends, key players, and noteworthy news in AI, individuals can gain valuable insights into the future of technology. Whether you are a professional in the tech industry, a business leader, or a student/researcher, understanding the advancements in AI is essential for staying ahead in the digital age. Subscribe to our newsletter for regular updates on Artificial Intelligence trends and advancements.

**Call to Action:** Stay informed on the latest trends and advancements in Artificial Intelligence by subscribing to our newsletter for regular updates.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: Exploring the Latest Trends in Technology

Outline:
I. Introduction
- Brief overview of the importance of technology in today's society
- Mention of the rapid pace of technological advancements
- Thesis statement highlighting the need to stay updated on the latest trends

II. Latest Trends in Technology
- Artificial Intelligence and Machine Learning
- Internet of Things (IoT)
- 5G Technology
- Virtual Reality and Augmented Reality
- Cybersecurity advancements

III. Key Players in the In

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).